DCGAN 성공한 버전. MNIST 데이터에 맞춰 공식 튜토리얼 약간 바꿨습니다.

In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
!pip install torchsummary
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
# manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [2]:
dataroot = './data/mnist'
workers = 2
ngpu = 1
batch_size = 128

image_size = 64
nc = 1
nz = 100
ngf = 256
ndf = 256

num_epochs = 200
lr = 0.0002
beta1 = 0.5

In [3]:
# dataset = dset.ImageFolder(root=dataroot,
#                            transform=transforms.Compose([
#                                transforms.Resize(image_size),
#                                transforms.CenterCrop(image_size),
#                                transforms.ToTensor(),
#                                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
#                            ]))

# dataloader = torch.utils.data.Dataloader(dataset,
#                                          batch_size=batch_size,
#                                          shuffle=True,
#                                          num_workers=workers)

transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize([0.5], [0.5])
])

dataset = dset.MNIST(root=dataroot,
                     train=True,
                     transform=transform,
                     download=True
                     )

dataloader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         num_workers=workers)

device = torch.device("cuda:0" if torch.cuda.is_available() and ngpu > 0 else "cpu")

print(len(dataloader))

# real_batch = next(iter(dataloader))
# plt.figure(figsize=(8, 8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64],
#                                          padding=2,
#                                          normalize=True).cpu(),
#                                          (1, 2, 0)))

469


In [4]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [5]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(  # nz
            nn.ConvTranspose2d(nz, ngf*4, 4, 1, 0, bias=False),  # ngf*4@4*4
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 3, 2, 1, bias=False),  # ngf*2@7*7
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),  # ngf@14*14
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),  # nc@28*28
            nn.Tanh()  # [-1, 1]
        )

    def forward(self, input):
        return self.main(input)

In [6]:
netG = Generator(ngpu).to(device)

if device.type == "cuda" and ngpu > 1:
    netG = nn.DataParallel(netG, list(range(ngpu)))

netG.apply(weights_init)
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(1024, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(256, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)


In [7]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(  # nc@28*28
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),  # ndf@14*14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),  # ndf*2@7*7
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, 1, 4, 1, 0, bias=False),  # 1@4*4
            nn.Flatten(1, -1),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.main(input)

In [8]:
netD = Discriminator(ngpu).to(device)

if device.type == "cuda" and ngpu > 1:
    netD = nn.DataParallel(netD, list(range(ngpu)))

netD.apply(weights_init)

print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (6): Flatten()
    (7): Linear(in_features=16, out_features=1, bias=True)
    (8): Sigmoid()
  )
)


In [9]:
summary(netG, (100, 1, 1))
summary(netD, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 1024, 4, 4]       1,638,400
       BatchNorm2d-2           [-1, 1024, 4, 4]           2,048
              ReLU-3           [-1, 1024, 4, 4]               0
   ConvTranspose2d-4            [-1, 512, 7, 7]       4,718,592
       BatchNorm2d-5            [-1, 512, 7, 7]           1,024
              ReLU-6            [-1, 512, 7, 7]               0
   ConvTranspose2d-7          [-1, 256, 14, 14]       2,097,152
       BatchNorm2d-8          [-1, 256, 14, 14]             512
              ReLU-9          [-1, 256, 14, 14]               0
  ConvTranspose2d-10            [-1, 1, 28, 28]           4,096
             Tanh-11            [-1, 1, 28, 28]               0
Total params: 8,461,824
Trainable params: 8,461,824
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw

In [10]:
criterion = nn.BCELoss()
d_optimizer = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
g_optimizer = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [11]:
def denorm(x):
    out = (x+1)/2
    return out

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [12]:
img_list = []
g_losses = []
d_losses = []
total_step = len(dataloader)

print("Starting Training Loop...")
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(dataloader, 0):
        """# train D
        # train with real images
        netD.zero_grad()
        real_cpu = data[0].to(device)
        print(real_cpu.shape)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.bool, device=device)
        output = netD(real_cpu).view(-1)

        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake images
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        
        errD_fake = criterion(output, label)
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()"""

        # train netD
        print(type(images))
        images = images.to(device)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        outputs = netD(images)
        d_loss_real = criterion(outputs, real_labels[:len(outputs)])
        real_score = outputs

        z = torch.randn(batch_size, nz, 1, 1).to(device)
        fake_images = netG(z)
        outputs = netD(fake_images)
        d_loss_fake = criterion(outputs, fake_labels[:len(outputs)])
        fake_score = outputs

        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()


        """# train G
        netG.zero_grad()
        label.fill(real_label)
        output = netD(fake).view(-1)

        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()"""

        # train netG
        z = torch.randn(batch_size, nz, 1, 1).to(device)
        fake_images = netG(z)
        outputs = netD(fake_images)

        g_loss = criterion(outputs, real_labels[:len(outputs)])
        reset_grad()
        g_loss.backward()
        g_optimizer.step()


        """# output training stats
        if i%200 == 0:
            print('[%d/%d][%d/%d]\tloss_D: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
            % (epoch+1, num_epochs, i, len(dataloader),
               errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if iters%500 == 0 or (epoch+1==num_epochs and i+1==len(dataloader)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
        iters += 1"""

        # output training data
        if (i+1)%200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch+1, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
        g_losses.append(g_loss.item())
        d_losses.append(d_loss.item())
    
    if not os.path.exists('./gan-02-images'):
        os.makedirs('./gan-02-images')

    if (epoch+1) == 1:
        vutils.save_image(denorm(images), './gan-02-images/real_images.png')
    
    vutils.save_image(denorm(fake_images), './gan-02-images/fake_images-{}.png'.format(epoch+1))
    img_list.append(vutils.make_grid(fake_images, padding=2, normalize=True))

Starting Training Loop...
<class 'torch.Tensor'>


RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`
Exception raised from createCublasHandle at ..\aten\src\ATen\cuda\CublasHandlePool.cpp:8 (most recent call first):
00007FFF176175A200007FFF17617540 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFEA5F9AEA800007FFEA5F99E70 torch_cuda.dll!at::cuda::getCurrentCUDASparseHandle [<unknown file> @ <unknown line number>]
00007FFEA5F9A7D800007FFEA5F99E70 torch_cuda.dll!at::cuda::getCurrentCUDASparseHandle [<unknown file> @ <unknown line number>]
00007FFEA5F9B66700007FFEA5F9B1A0 torch_cuda.dll!at::cuda::getCurrentCUDABlasHandle [<unknown file> @ <unknown line number>]
00007FFEA5F9B24700007FFEA5F9B1A0 torch_cuda.dll!at::cuda::getCurrentCUDABlasHandle [<unknown file> @ <unknown line number>]
00007FFEA5F9320700007FFEA5F924B0 torch_cuda.dll!at::native::sparse_mask_cuda [<unknown file> @ <unknown line number>]
00007FFEA549CA9700007FFEA549B990 torch_cuda.dll!at::native::lerp_cuda_tensor_o

In [ ]:
torch.save(netG.state_dict(), 'G.ckpt')
torch.save(netD.state_dict(), 'D.ckpt')